# Workshop Microsoft - Sostenibilidad
En el siguiente ejemplo analizaremos el coste energético de un modelo de la inferencia del modelo Phi de Microsoft.
Se puede realizar este ejemplo con CPU o GPU. En caso de ser con GPU es necesario indicarlo en los parámetros de configuración

In [ ]:
%pip install transformers==4.48 accelerate sentencepiece tokenizer codecarbon -q
%pip install dash dash-bootstrap-components python-dotenv -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Posiblemente, sea necesario reiniciar el kernel

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import textwrap
from codecarbon import EmissionsTracker
from codecarbon import track_emissions
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
# parametros de configuración
model_name = "microsoft/Phi-3.5-mini-instruct"
project_name="worshop-ia-sostenible"

#TODO CAMBIAR RUTA DEL FICHERO
#output_dir_emissions_track="/lakehouse/default/Files"
output_dir_emissions_track="/home/azureuser/cloudfiles/code/Users/paul.vanb/worshop-ia-sostenible/outputs"
output_file_emissions_track=f"{project_name}_emissions.csv"

output_filename = os.path.join(output_dir_emissions_track,output_file_emissions_track )
device_map="auto"
cloud_region="westus2"

In [ ]:

def wrap_text(text, width=100):
    """
    Envuelve el texto dado para que cada línea no supere el ancho especificado.
    
    Args:
        text (str): El texto de entrada.
        width (int): El ancho máximo de cada línea (por defecto, 100 caracteres).
    
    Returns:
        str: Texto con saltos de línea insertados para cumplir con el ancho especificado.
    """
    lines = text.split('\n')  # Divide el texto en líneas basadas en saltos de línea existentes.
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]  # Envuelve cada línea individualmente.
    wrapped_text = '\n'.join(wrapped_lines)  # Une las líneas envueltas con saltos de línea.
    return wrapped_text

def generate(pipe, input_text, system_prompt="", **generation_args):
    """
    Genera una respuesta basada en el texto de entrada utilizando un modelo de IA.
    
    Args:
        input_text (str): Texto de entrada del usuario.
        system_prompt (str): Mensaje de sistema opcional para guiar la respuesta del modelo.
        max_length (int): Longitud máxima de la respuesta generada (por defecto, 1024 caracteres).
    
    Returns:
        str: Texto generado y formateado con saltos de línea adecuados.
    """
    # Si no se proporciona un mensaje del sistema, se usa un mensaje por defecto.
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    
    # Construcción del historial de mensajes con el mensaje del sistema y la entrada del usuario.
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_text},
    ]
    
    # Generación de la respuesta del modelo.
    output = pipe(messages, **generation_args)
    text = output[0]['generated_text']  # Obtiene el texto generado.
    
    # Envuelve el texto generado para cumplir con el ancho especificado.
    wrapped_text = wrap_text(text)
    return wrapped_text

@track_emissions(project_name="workshop-sostenibilidad", output_dir=output_dir_emissions_track,
                 cloud_provider="azure", cloud_region= cloud_region, pue=1.19)
def answer_to_different_messages(pipe,messages):
    """
    Procesa una lista de mensajes y genera respuestas mientras realiza un seguimiento de las emisiones.
    
    Args:
        messages (list): Lista de mensajes de entrada.
    
    Returns:
        list: Lista de respuestas generadas junto con los mensajes de entrada y sus identificadores.
    """
    outputs = []  # Lista para almacenar los resultados.
    system_prompt = "Eres un asistente de Microsoft experto en sostenibilidad llamado SostAI"  # Mensaje del sistema.
    
    # Itera sobre cada mensaje y genera una respuesta.
    for mid, message in enumerate(messages):
        output = generate(pipe, message, system_prompt=system_prompt)  # Genera la respuesta.
        outputs.append([mid, message, output])  # Almacena el identificador, mensaje y respuesta.
    
    return outputs  # Devuelve la lista de respuestas generadas.

[codecarbon INFO @ 17:38:09] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.092193603515625 W
[codecarbon INFO @ 17:38:09] Energy consumed for all CPUs : 0.000407 kWh. Total CPU Power : 97.5 W
[codecarbon INFO @ 17:38:09] 0.000419 kWh of electricity used since the beginning.


# Carga del Modelo
Usaremos la librería transformers de Hugginface para cargar el modelo de microsoft Phi-3.5 mini instruído.


In [ ]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map= device_map,
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.79it/s]


# Ejercicio 2
Ahora vamos a hacer lo mismo que en el caso anterior, peri cambiando la región a brasil sur.
Para identificar el nombre del centro de datos, usa: https://datacenters.microsoft.com/globe/explore

In [ ]:
#%%time
# Start tracking
tracker = EmissionsTracker(tracking_mode="process",
                           output_dir=output_dir_emissions_track) 
tracker.start()


emmissions = tracker.stop()


[codecarbon INFO @ 09:47:45] Codecarbon is taking the configuration from global file: /home/azureuser/.codecarbon.config
[codecarbon INFO @ 09:47:45] [setup] RAM Tracking...
[codecarbon INFO @ 09:47:45] [setup] CPU Tracking...
[codecarbon DEBUG @ 09:47:45] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Platform not supported by Intel Power Gadget
[codecarbon DEBUG @ 09:47:45] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Intel RAPL files not found at /sys/class/powercap/intel-rapl on linux
[codecarbon DEBUG @ 09:47:45] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 09:47:45] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon DEBUG @ 09:47:46] CPU : We detect a Intel(R) Xeon(R) Platinum 8272CL CP

CPU times: user 58.1 s, sys: 120 ms, total: 58.2 s
Wall time: 35.2 s


In [ ]:
print(f"All hardware identified:{tracker._hardware}")
print(f"GPU/CPU Power: {tracker._hardware[1].total_power()}")
print(f"Hardware zone: {tracker._geo}")
print(f"Emissions from this training run: {emmissions:5f} kg CO2eq")

All hardware identified: [RAM(), CPU(Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz > 195.0W)]
GPU/CPU Power: Power(kW=0.0975)
Hardware zone: GeoMetadata(country_iso_code=USA, country_name=United States, region=washington)
Emissions from this training run: 0.000069 kg CO2eq


# Ejercicio 2 (opcional)
Vamos a observar las diferencias en coste energético entre cpu y gpu, para eso vamos a ejecutar con una lista de 20 mensajes a tu elección, la función answer_to_different_messages con gpu y luego cpu.
Para luego diferenciarlas en el output, asigna a la primera project_name "phi-emissions-cpu" y a la segunda "phi-emissions-gpu".
Chequea en cada ejecución, que tracker._hardware es gpu y luego cpu. Tendrás que reiniciar el notebook.

Observa los resultados y analizalos.